#                                    --> PIPELINE PROJECT <--

In [1]:
# importar modulos necesarios

import pandas as pd
import numpy as np
import re

1. Acquisition

In [2]:
# cargar el dataset elegido

avocados = pd.read_csv('avocado.csv')

avocados.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [3]:
avocados.shape

(18249, 14)

2. Wrangling

In [4]:
# comprobar los datos nulos del data set

avocados.isnull().sum()  # No hay nulos en ninguna columna

Unnamed: 0      0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

In [5]:
avocados.columns

Index(['Unnamed: 0', 'Date', 'AveragePrice', 'Total Volume', '4046', '4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type',
       'year', 'region'],
      dtype='object')

In [6]:
# Comprobar que no hay duplicados
before = len(avocados)

select_columns = ['Date', 'AveragePrice', 'Total Volume', '4046', '4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type',
       'year', 'region']

avocados = avocados[select_columns].drop_duplicates()

after = len(avocados)

print('Numero de registros duplicados: ', str(before - after))


Numero de registros duplicados:  0


In [7]:
# Extraer el mes de la columna fecha y desechar el resto

month = []
for e in avocados["Date"]:
    aux= e.split("-")
    month.append(aux[1])
    
avocados['month'] = pd.Series(month, index=avocados.index) # Añadir columna Month al dataframe


In [8]:
# Cambio del orden de las columnas

column_order = ['region','year','month','type','AveragePrice','Total Volume','4046','4225',
       '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']

avocados = avocados[column_order]

In [9]:
# comprobar el tipo de datos de las columnas. cambiar year a objeto

avocados['year'] = avocados['year'].astype('object') 

In [10]:
# renombrar algunas columnas

avocados.columns = ['Region','Year','Month','Variety','Avg Price','Total Volume',
                    'S-M size','L size','XL size','Total Bags','S bags','L bags','XL bags']
avocados.head()

,Region,Year,Month,Variety,Avg Price,Total Volume,S-M size,L size,XL size,Total Bags,S bags,L bags,XL bags
0,Albany,2015,12,conventional,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0
1,Albany,2015,12,conventional,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0
2,Albany,2015,12,conventional,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0
3,Albany,2015,12,conventional,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0
4,Albany,2015,11,conventional,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0


In [11]:
# Guardo un csv con los datos manipulados

avocados.to_csv('manipulated_avocados.csv',index=False)

3.Analysis

HIPÓTESIS:
- La variedad de aguacate convencional tendrá un precio medio inferior al aguacate orgánico.
- Las regiones donde el precio medio sea más alto presentarán un volument total de venta menor
- El precio medio será menor el més en el que se venda un mayor volumen de aguacates
- El % que aporten al volumen total (aguacates S-M, aguacates L, aguacares XL y bolsas (mezcla tamaños) tendrán relación con el precio medio.


In [22]:
# Dado que del año 2018 solo hay registros hasta marzo voy a eliminar esos datos
filteredyear = list(avocados[(avocados['Year']==2018)].index) 
avocados = avocados.drop(filteredyear, axis=0)

# Las regiones vienen reflejadas todas juntas de varias maneras: Total US, grandes regiones y por estados.
# Voy a separar en 3 dataframe: Uno de regiones, otro del total y otro de ciudades

avocados_region = avocados[(avocados['Region'] == 'Plains')|(avocados['Region'] =='West')|
                    (avocados['Region'] =='California')|(avocados['Region'] =='SouthCentral')|
                    (avocados['Region'] =='Northeast')|(avocados['Region'] =='Southeast')|
                    (avocados['Region'] =='GreatLakes')|(avocados['Region'] =='Midsouth')]
avocados_totalUS = avocados[(avocados['Region'] == 'TotalUS')]

avocados_cites= avocados[(avocados['Region'] != 'Plains')|(avocados['Region'] !='West')|
                    (avocados['Region'] !='California')|(avocados['Region'] !='SouthCentral')|
                    (avocados['Region'] !='Northeast')|(avocados['Region'] !='Southeast')|
                    (avocados['Region'] !='GreatLakes')|(avocados['Region'] !='Midsouth')|
                   (avocados['Region'] != 'TotalUS')]

In [19]:
#VARIEDAD vs AVG PRICE (En todo US)

variedad_precio =avocados_totalUS.groupby(['Variety','Year'])['Avg Price'].describe()
variedad_precio # Efectivamente el aguacate orgánico es más caro que el convencional. 

count      mean       std   min    25%    50%     75%   max
Variety      Year                                                             
conventional 2015   52.0  1.012500  0.047481  0.89  0.980  1.020  1.0400  1.09
             2016   52.0  1.046731  0.163692  0.76  0.930  1.000  1.1500  1.44
             2017   53.0  1.221698  0.205705  0.77  1.090  1.210  1.3200  1.65
organic      2015   52.0  1.502308  0.214061  1.00  1.445  1.525  1.6450  1.80
             2016   52.0  1.480385  0.138535  1.25  1.410  1.450  1.5525  1.93
             2017   53.0  1.651509  0.226201  1.16  1.480  1.680  1.8200  2.09

In [21]:
variedad_precio_region = pd.pivot_table(avocados_region, values=['Avg Price'], index=['Region'], columns=['Year','Variety'])

variedad_precio_region

Avg Price                                                \
Year                 2015                   2016                   2017   
Variety      conventional   organic conventional   organic conventional   
Region                                                                    
California       1.020385  1.611346     1.046154  1.613654     1.253019   
GreatLakes       1.077692  1.574808     1.141923  1.412115     1.331321   
Midsouth         1.119615  1.583077     1.185962  1.515769     1.325472   
Northeast        1.214808  1.872692     1.319423  1.871923     1.505283   
Plains           1.077308  1.798462     1.127692  1.627308     1.301509   
SouthCentral     0.812115  1.411923     0.858269  1.195577     0.950943   
Southeast        1.084231  1.745192     1.071154  1.502692     1.334151   
West             0.940000  1.535577     0.915769  1.445962     1.098302   

                        
Year                    
Variety        organic  
Region                  
California    1.814717  
GreatLakes    1.517925  
Midsouth      1.715660  
Northeast     1.882830  
Plains        1.719434  
SouthCentral  1.374717  
Southeast     1.690189  
West          1.681698

In [24]:
# Mes con mas volumen de ventas
month_volume = avocados_totalUS.groupby(['Month'])['Total Volume'].sum()

month_volume

# Mayo es el mes con mayor volumen de ventas, seguido de enero.

Month
01    5.010337e+08
02    4.772438e+08
03    4.421213e+08
04    4.677807e+08
05    5.552051e+08
06    4.543907e+08
07    4.918025e+08
08    4.263096e+08
09    3.685976e+08
10    3.877041e+08
11    3.589926e+08
12    4.099263e+08
Name: Total Volume, dtype: float64

In [27]:
year_month_volume = pd.pivot_table(avocados_region, values=['Total Volume'], index=['Month'], columns=['Year'])

year_month_volume

Total Volume                            
Year           2015          2016          2017
Month                                          
01     1.882086e+06  2.238978e+06  2.518274e+06
02     2.148491e+06  2.524482e+06  2.783961e+06
03     1.984466e+06  2.307607e+06  2.119955e+06
04     2.039883e+06  2.374001e+06  2.316152e+06
05     2.284402e+06  2.647677e+06  2.509981e+06
06     2.254751e+06  2.421648e+06  2.423456e+06
07     2.110763e+06  2.209482e+06  2.249438e+06
08     1.977398e+06  2.185615e+06  2.003725e+06
09     1.941528e+06  2.136636e+06  1.681173e+06
10     1.800485e+06  1.697194e+06  1.708719e+06
11     1.733995e+06  1.529784e+06  1.911981e+06
12     1.741298e+06  1.952924e+06  2.168701e+06

In [28]:
year_avgprice = avocados_totalUS.groupby(['Year'])['Avg Price'].mean()
year_avgprice

# Por año se observa que se ha incrementado el precio medio en 2017. Aunque no es lineal, en 2016 disminuyó

Year
2015    1.257404
2016    1.263558
2017    1.436604
Name: Avg Price, dtype: float64

In [29]:
region_volume = avocados_region.groupby(['Region'])['Total Volume'].sum()

region_volume.sort_values(ascending=False)

Region
West            9.943021e+08
California      9.452403e+08
SouthCentral    9.199759e+08
Northeast       6.469574e+08
Southeast       5.549539e+08
GreatLakes      5.364819e+08
Midsouth        4.600360e+08
Plains          2.831605e+08
Name: Total Volume, dtype: float64